In [1]:
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import torch
import torch.utils.data as dutils

import torchvision.datasets as dataset
import torchvision.transforms as dtrans

In [6]:
dataroot = '../data/'
device = 'cpu'
seed = 99

In [7]:
input_dim = 784
output_dim = 10
num_examples = 60000

batch_size = 32

torch.manual_seed(seed);

### Data setup

In [10]:
transform = dtrans.Compose([dtrans.Grayscale(num_output_channels=1),
                            dtrans.Resize((28,28)),
                            dtrans.ToTensor(), dtrans.Normalize((0.5,), (0.5,))])
train_set = dataset.CIFAR10(root=dataroot, download=False, train=True, transform=transform)
test_set = dataset.CIFAR10(root=dataroot, download=False, train=False, transform=transform)

train_loader = dutils.DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = dutils.DataLoader(test_set, batch_size=batch_size, shuffle=False)

### Model setup

In [106]:
def l2_penalty(weight):
    penalty = torch.pow(weight, 2)
    penalty = torch.sqrt(penalty).mean()
    return penalty        

In [58]:
def cross_entropy(yhat, y):
    probs = torch.gather(yhat, 1, y.view(-1, 1))
    loss = -torch.log(probs + 1e-6)
    return loss.mean()

In [59]:
def softmax(y_linear):
    row_max = y_linear.max(dim=1)[0]
    row_max.unsqueeze_(1)
    row_max = row_max.repeat(1, y_linear.shape[1])
    exp = torch.exp(y_linear - row_max)
    norm = exp.sum(dim=1).unsqueeze(1).repeat(1, y_linear.shape[1])
    return exp / norm

In [9]:
# sample_y_linear = torch.randn(10, 10)
# sample_yhat = softmax(sample_y_linear)
# print(sample_y_linear)
# print(sample_yhat)

In [60]:
def SGD(params, lr):
    for param in params:
        param.data = param.data - lr * param.grad

In [ ]:
class Net():
    def __init__(self, input_dim, out_dim, device):
        w = torch.randn(size=(input_dim, output_dim), device=device)
        b = torch.randn(output_dim, device=device)
        self.w = w / torch.norm(w)
        self.b = b /torch.norm(b)
        self.w.requires_grad = True
        self.b.requires_grad = True
#         print('Weight shape: ', self.w.shape)
#         print('Bias shape: ', self.b.shape)
        
    def forward(self, x):
        w = self.w
        bias = self.b
        out = torch.mm(x, w) + bias
        out = softmax(out)
        return out.squeeze()
    
    def parameters(self):
        return [self.w, self.b]
    
    def zero_grad(self):
        self.w.grad = None
        self.b.grad = None
        
    def __call__(self, x):
        return self.forward(x)

In [30]:
def test(net, test_loader, device):
    loss = 0
    error = 0
    for i, (images, labels) in enumerate(test_loader):
        images, labels = images.to(device), labels.to(device)
        images = images.view(-1, input_dim)
        with torch.no_grad():
            out = net(images)
            loss += cross_entropy(out, labels).item()
            error += (out.argmax(1) != labels).sum().item()
    loss /= i       
    print('Loss: %.4f, Error: %d' % (loss, error))

In [110]:
def train(net, train_loader, device):
    params = net.parameters()
    lr = 0.1
    
    loss = 0
    error = 0
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        images = images.view(-1, 784)
        
        out = net(images)
        loss_batch = cross_entropy(out, labels)
        loss_batch = loss_batch + 1 * l2_penalty(net.w)
        
        net.zero_grad()
        loss_batch.backward()
#         SGD(params, lr)
        opt.step()
        
        loss += loss_batch.item()
        error += (out.argmax(1) != labels).sum().item()
    loss /= i       
    print('Loss: %.4f, Error: %d' % (loss, error))

In [111]:
net = Net(input_dim, output_dim, device)
opt = torch.optim.Adam(net.parameters(), lr=0.003)

test(net, test_loader, device)
print l2_penalty(net.w)
for epochs in range(50):
    train(net, train_loader, device)
    test(net, test_loader, device)
    print l2_penalty(net.w)

Loss: 2.3558, Error: 8961
tensor(0.0091, grad_fn=<MeanBackward1>)
Loss: 2.1695, Error: 37418
Loss: 2.1325, Error: 7418
tensor(0.0215, grad_fn=<MeanBackward1>)
Loss: 2.1481, Error: 36898
Loss: 2.1299, Error: 7288
tensor(0.0242, grad_fn=<MeanBackward1>)
Loss: 2.1436, Error: 36611
Loss: 2.1603, Error: 7404
tensor(0.0246, grad_fn=<MeanBackward1>)
Loss: 2.1409, Error: 36761
Loss: 2.1711, Error: 7541
tensor(0.0262, grad_fn=<MeanBackward1>)
Loss: 2.1464, Error: 36714
Loss: 2.1356, Error: 7285
tensor(0.0268, grad_fn=<MeanBackward1>)
Loss: 2.1485, Error: 36786
Loss: 2.1293, Error: 7520
tensor(0.0265, grad_fn=<MeanBackward1>)
Loss: 2.1412, Error: 36620
Loss: 2.1223, Error: 7362
tensor(0.0270, grad_fn=<MeanBackward1>)
Loss: 2.1407, Error: 36569
Loss: 2.1257, Error: 7385
tensor(0.0273, grad_fn=<MeanBackward1>)
Loss: 2.1410, Error: 36575
Loss: 2.2046, Error: 7588
tensor(0.0281, grad_fn=<MeanBackward1>)
Loss: 2.1421, Error: 36706
Loss: 2.1973, Error: 7545
tensor(0.0282, grad_fn=<MeanBackward1>)
Loss

KeyboardInterrupt: 

In [64]:
a = torch.randn(10, 5)
b = (torch.rand(10)*5).long()

print a
print b.view(-1, 1)

# torch.gather(a, 1, b.view(-1, 1))


tensor([[-1.4813,  0.3770,  1.1661,  1.1120,  0.7396],
        [-1.1656,  0.3580, -1.0823,  1.3389,  0.7177],
        [ 0.8475, -1.3822, -0.1256, -0.7188, -1.2978],
        [ 1.3189,  1.2414, -0.3685,  0.1967, -0.3643],
        [-0.6959,  0.3633, -0.3550, -0.6447, -0.6605],
        [-0.9395,  0.2178, -2.1892, -1.5369,  1.2575],
        [ 2.0529, -1.3377, -1.8684, -0.3506, -0.3648],
        [ 0.1550,  1.9691, -0.3749, -0.4108,  0.0268],
        [ 0.0091, -0.1698,  0.2955,  0.1400, -0.7759],
        [-0.7668,  0.9081,  0.9108, -2.1320,  1.0144]])
tensor([[0],
        [2],
        [0],
        [2],
        [0],
        [4],
        [0],
        [4],
        [2],
        [3]])
